In [2]:
import os
import numpy as np
import time
import tensorflow as tf
from sklearn.model_selection import train_test_split

seed = 11068 
np.random.seed(seed) ## Use your own unique seed
tf.random.set_seed(seed) ## Use your own unique seed

In [ ]:
## TODO
## Load Data
##for.npy files

X_train = np.load("x_train.npy").reshape(-1, 28*28)
X_test = np.load("x_test.npy").reshape(-1, 28*28)
## Create Validation data from train splits
X_val = None
Y_train = np.load("y_train.npy")
Y_test = np.load("y_test.npy")
Y_val = None
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=0)

In [3]:
## TODO
## Load Data
## For .npz files

#-----> This code is for loading .npz files <--------#

X_train_data = np.load("train-imgs.npz")
X_train = [X_train_data[key] for key in X_train_data.keys()]
X_train = np.concatenate(X_train, axis = 0).reshape(-1, 28*28)

X_test_data = np.load("test-imgs.npz")
X_test = [X_test_data[key] for key in X_test_data.keys()]
X_test = np.concatenate(X_test, axis = 0).reshape(-1, 28*28)

## Create Validation data from train splits
X_val = None
Y_train_data = np.load("train-labels.npz")
Y_train = [Y_train_data[key] for key in Y_train_data.keys()]
Y_train = np.concatenate(Y_train, axis = 0)

Y_test_data = np.load("test-labels.npz")
Y_test = [Y_test_data[key] for key in Y_test_data.keys()]
Y_test = np.concatenate(Y_test, axis = 0)

Y_val = None
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=0)

In [4]:
## Define model specific parameters
## Repalce None with values
size_input = 28*28
size_hidden1 = 128
size_hidden2 = 256
size_hidden3 = 128
size_output = 10
number_of_train_examples = 48000
number_of_dev_examples = 12000
number_of_test_examples = 10000
batch_size = 1000
NUM_EPOCHS = 20

In [5]:
## TODO
# Normalize your X
# Convert your labels into one-hot encoding
# Hint look into function tf.keras.utils.to_categorical or Other function is tf.one_hot()

def normalize_func(X):
    X = (X - np.min(X))/(np.max(X) - np.min(X))
    return X

# Apply the normalization function to each example in the dataset
X_train = normalize_func(X_train)
X_val = normalize_func(X_val)
X_test = normalize_func(X_test)


Y_train = normalize_func(Y_train)
Y_val = normalize_func(Y_val)
Y_test = normalize_func(Y_test)

# Convert your labels into one-hot encoding

Y_train = tf.cast(Y_train, dtype=tf.int64)
Y_val = tf.cast(Y_val, dtype=tf.int64)

Y_train = tf.one_hot(Y_train, depth = 10)

Y_val = tf.one_hot(Y_val, depth = 10)

Y_test = tf.one_hot(Y_test, depth = 10)

For L1 Regularization

In [6]:
# Define class to build mlp model
class MLP(object):
 def __init__(self, size_input, size_hidden1, size_hidden2, size_hidden3, size_output, device=None):
    """
    size_input: int, size of input layer
    size_hidden1: int, size of the 1st hidden layer
    size_hidden2: int, size of the 2nd hidden layer
    size_output: int, size of output layer
    device: str or None, either 'cpu' or 'gpu' or None. If None, the device to be used will be decided automatically during Eager Execution
    """
    self.size_input, self.size_hidden1, self.size_hidden2, self.size_hidden3, self.size_output, self.device =\
    size_input, size_hidden1, size_hidden2, size_hidden3, size_output, device
    
    ### TODO
    ### Declare all your variables
    self.W1 = tf.Variable(tf.random.normal([self.size_input, self.size_hidden1]))
    self.W2 = tf.Variable(tf.random.normal([self.size_hidden1, self.size_hidden2]))
    self.W3 = tf.Variable(tf.random.normal([self.size_hidden2, self.size_hidden3]))
    self.W4 = tf.Variable(tf.random.normal([self.size_hidden3, self.size_output]))

    self.b1 = tf.Variable(tf.zeros([1, self.size_hidden1]))
    self.b2 = tf.Variable(tf.zeros([1, self.size_hidden2]))
    self.b3 = tf.Variable(tf.zeros([1, self.size_hidden3]))
    self.b4 = tf.Variable(tf.zeros([1, self.size_output]))

    
    # Define variables to be updated during backpropagation
    self.variables = [self.W1, self.W2, self.W3, self.W4, self.b1, self.b2, self.b3, self.b4]
  
 def forward(self, X):
    """
    forward pass
    X: Tensor, inputs
    """
    if self.device is not None:
      with tf.device('gpu:0' if self.device=='gpu' else 'cpu'):
        self.y = self.compute_output(X)
    else:
      self.y = self.compute_output(X)
      
    return self.y

 def loss(self, y_pred, y_true):
    '''
    y_pred - Tensor of shape (batch_size, size_output)
    y_true - Tensor of shape (batch_size, size_output)
    '''
    ## TODO

    # Convert your pred and true to tf.float32
    # Ensure your shapes are (batch_size, size_output)
    y_true_tf = tf.cast(tf.reshape(y_true, (-1, self.size_output)), dtype=tf.float32)
    y_pred_tf = tf.cast(y_pred, dtype=tf.float32)

    cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

    # This above function will add softmax to your final layer and then perform cross-entropy
    loss_x = cce(y_true_tf, y_pred_tf)

    # TODO
    # Modify above loss and add L1 regularization

    l1_lamda = 0.001
    l1_reg = l1_lamda * (tf.reduce_sum(tf.abs(self.W1)) + tf.reduce_sum(tf.abs(self.W2)) + tf.reduce_sum(tf.abs(self.W3)) + tf.reduce_sum(tf.abs(self.W4)))
    loss = l1_reg + loss_x
    return loss

 def backward(self, X_train, y_train):
    """
    backward pass
    """
    lr = 0.001 #Play with your learning rate
    optimizer = tf.keras.optimizers.RMSprop(learning_rate = lr) # Play with various Optimizers and pick the one that works best for your design
    
    with tf.GradientTape() as tape:
        
      predicted = self.forward(X_train)
      current_loss = self.loss(predicted, y_train)
        
    grads = tape.gradient(current_loss, self.variables)
    optimizer.apply_gradients(zip(grads, self.variables)) # Optimizer
           
 def compute_output(self, X):
    """
    Custom method to obtain output tensor during forward pass
    """
    # Cast X to float32
    X_tf = tf.cast(X, dtype=tf.float32)

    ## TODO
    # Compute values in hidden layers
    H1 = tf.matmul(X_tf, self.W1) + self.b1
    Z1 = tf.nn.sigmoid(H1)

    H2 = tf.matmul(Z1, self.W2) + self.b2
    Z2 = tf.nn.sigmoid(H2)

    H3 = tf.matmul(Z2, self.W3) + self.b3
    Z3 = tf.nn.sigmoid(H3)

    H4 = tf.matmul(Z3, self.W4) + self.b4
    # Compute output
    output = H4 # Logits
    
    # Remember the loss function has keras loss objective, that explicitly applies softmax over logits
    return (output)

 For L2 Regularization

In [ ]:
# Define class to build mlp model
class MLP(object):
 def __init__(self, size_input, size_hidden1, size_hidden2, size_hidden3, size_output, device=None):
    """
    size_input: int, size of input layer
    size_hidden1: int, size of the 1st hidden layer
    size_hidden2: int, size of the 2nd hidden layer
    size_output: int, size of output layer
    device: str or None, either 'cpu' or 'gpu' or None. If None, the device to be used will be decided automatically during Eager Execution
    """
    self.size_input, self.size_hidden1, self.size_hidden2, self.size_hidden3, self.size_output, self.device =\
    size_input, size_hidden1, size_hidden2, size_hidden3, size_output, device
    
    ### TODO
    ### Declare all your variables
    self.W1 = tf.Variable(tf.random.normal([self.size_input, self.size_hidden1]))
    self.W2 = tf.Variable(tf.random.normal([self.size_hidden1, self.size_hidden2]))
    self.W3 = tf.Variable(tf.random.normal([self.size_hidden2, self.size_hidden3]))
    self.W4 = tf.Variable(tf.random.normal([self.size_hidden3, self.size_output]))

    self.b1 = tf.Variable(tf.zeros([1, self.size_hidden1]))
    self.b2 = tf.Variable(tf.zeros([1, self.size_hidden2]))
    self.b3 = tf.Variable(tf.zeros([1, self.size_hidden3]))
    self.b4 = tf.Variable(tf.zeros([1, self.size_output]))

    
    # Define variables to be updated during backpropagation
    self.variables = [self.W1, self.W2, self.W3, self.W4, self.b1, self.b2, self.b3, self.b4]
  
 def forward(self, X):
    """
    forward pass
    X: Tensor, inputs
    """
    if self.device is not None:
      with tf.device('gpu:0' if self.device=='gpu' else 'cpu'):
        self.y = self.compute_output(X)
    else:
      self.y = self.compute_output(X)
      
    return self.y

 def loss(self, y_pred, y_true):
    '''
    y_pred - Tensor of shape (batch_size, size_output)
    y_true - Tensor of shape (batch_size, size_output)
    '''
    ## TODO

    # Convert your pred and true to tf.float32
    # Ensure your shapes are (batch_size, size_output)
    y_true_tf = tf.cast(tf.reshape(y_true, (-1, self.size_output)), dtype=tf.float32)
    y_pred_tf = tf.cast(y_pred, dtype=tf.float32)

    cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

    # This above function will add softmax to your final layer and then perform cross-entropy
    loss_x = cce(y_true_tf, y_pred_tf)

    # TODO
    # Modify above loss and add  L2 regularization

    l2_lamda = 0.001
    l2_reg = l2_lamda * (tf.reduce_sum(tf.square(self.W1)) + tf.reduce_sum(tf.square(self.W2)) + tf.reduce_sum(tf.square(self.W3)) + tf.reduce_sum(tf.square(self.W4)))
    loss = l2_reg + loss_x
    return loss

 def backward(self, X_train, y_train):
    """
    backward pass
    """
    lr = 0.001 #Play with your learning rate
    optimizer = tf.keras.optimizers.RMSprop(learning_rate = lr) # Play with various Optimizers and pick the one that works best for your design
    
    with tf.GradientTape() as tape:
        
      predicted = self.forward(X_train)
      current_loss = self.loss(predicted, y_train)
        
    grads = tape.gradient(current_loss, self.variables)
    optimizer.apply_gradients(zip(grads, self.variables)) # Optimizer
           
 def compute_output(self, X):
    """
    Custom method to obtain output tensor during forward pass
    """
    # Cast X to float32
    X_tf = tf.cast(X, dtype=tf.float32)

    ## TODO
    # Compute values in hidden layers
    H1 = tf.matmul(X_tf, self.W1) + self.b1
    Z1 = tf.nn.sigmoid(H1)

    H2 = tf.matmul(Z1, self.W2) + self.b2
    Z2 = tf.nn.sigmoid(H2)

    H3 = tf.matmul(Z2, self.W3) + self.b3
    Z3 = tf.nn.sigmoid(H3)

    H4 = tf.matmul(Z3, self.W4) + self.b4
    # Compute output
    output = H4 # Logits
    
    # Remember the loss function has keras loss objective, that explicitly applies softmax over logits
    return (output)

In [ ]:
# Initialize model using GPU
mlp_on_gpu = MLP(size_input, size_hidden1, size_hidden2, size_hidden3, size_output, device='gpu')

time_start = time.time()

for epoch in range(NUM_EPOCHS):
    
  loss_total = tf.zeros([1,1], dtype=tf.float32)
  lt = 0
    
  train_ds = tf.data.Dataset.from_tensor_slices((X_train, Y_train)).shuffle(25, seed=epoch*(seed)).batch(batch_size)
  kz = 0
  accuracy_z = 0.0
  cur_train_acc = 0.0
  for inputs, outputs in train_ds:
    qw, tr = tf.shape(inputs)
    kz = kz + 1
    preds = mlp_on_gpu.forward(inputs) 
    loss_total = loss_total + mlp_on_gpu.loss(preds, outputs)
    lt = lt + mlp_on_gpu.loss(preds, outputs)
    mlp_on_gpu.backward(inputs, outputs)

  preds = mlp_on_gpu.forward(X_train)
  # Get probs, remember we only have logits from our forward function, we need to apply softmax on top of it to get probs
  preds = tf.nn.softmax(preds)
  correct_prediction = tf.equal(tf.argmax(preds, 1), tf.argmax(Y_train, 1))
  accuracy_z = accuracy_z + tf.reduce_mean(tf.cast(correct_prediction, "float"))
  cur_train_acc += accuracy_z.numpy()
  ds = cur_train_acc
  print('\nTrain Accuracy: {:.4f}'.format(ds))
  print('Number of Epoch = {} - Average Cross Entropy:= {} '.format(epoch + 1, np.sum(loss_total) / X_train.shape[0]))
  
  ## TODO
  cur_val_acc = 0.0
  accuracy_z = 0.0

  probs = mlp_on_gpu.forward(X_val)
  ## Now calculate Validation Accuracy
  probs = tf.nn.softmax(probs)
  correct_prediction = tf.equal(tf.argmax(probs, 1), tf.argmax(Y_val, 1))
  accuracy_z = tf.reduce_mean(tf.cast(correct_prediction, "float"))
  cur_val_acc = accuracy_z.numpy()


  

  print('\nValidation Accuracy: {:.4f}'.format(cur_val_acc))
  


        
time_taken = time.time() - time_start
    

    


print('\nTotal time taken (in seconds): {:.2f}'.format(time_taken))
#For per epoch_time = Total_Time / Number_of_epochs

In [ ]:
## TODO - Write a inference code and perform only one pass through test data
test_loss_total = tf.Variable(0, dtype=tf.float32)
test_loss_total = 0.0
test_ds = tf.data.Dataset.from_tensor_slices((X_test, Y_test)).shuffle(25, seed=epoch*(seed)).batch(batch_size)
  
  
for inputs, outputs in test_ds:
  preds = mlp_on_gpu.forward(inputs)
  preds = tf.nn.softmax(preds)
  test_loss_total = test_loss_total + mlp_on_gpu.loss(preds, outputs)
    

print('Test MSE: {:.4f}'.format(np.sum(test_loss_total.numpy()) / X_train.shape[0]))

probs = mlp_on_gpu.forward(X_test)
probs = tf.nn.softmax(probs)
correct_prediction = tf.equal(tf.argmax(probs, 1), tf.argmax(Y_test, 1))
accuracy_z = tf.reduce_mean(tf.cast(correct_prediction, "float"))

print('\nTesting Accuracy: {:.4f}'.format(accuracy_z))